In [3]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])

# input을 받아줘야하기 때문에 무시하기 위해서 _라는 argument 입력
def load_memory(_):
    return memory.load_memory_variables({})["chat_history"]


chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm


def invoke_chain(question):
    result = chain.invoke(
        {
            "question": question,
        }
    )
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)


# chain.invoke({
#     # RunnablePassthrough로 대체가능
#     # "chat_history": memory.load_memory_variables({})["chat_history"],
#     "question": "My name is Jingoo"
# })

In [4]:
invoke_chain("My name is Jingoo")

content='Hello Jingoo! How can I assist you today?'


In [5]:
invoke_chain("I live in Daejeon")

content="That's great to know! Is there anything specific you would like to know or talk about regarding Daejeon or anything else? Feel free to ask!"


In [6]:
invoke_chain("What's my name?")

content='Your name is Jingoo. How can I assist you further, Jingoo?'
